Welcome to the capy notebook! This file is meant as a guide to various scores to measure clustering between populations in a region with sub regions.

See our main literature for more explanations of the functions, their motivations and histories, and other applications. Below is a brief summary of the functions.

Suppose that there are $2$ populations of interest in a region, populations $X$ and $Y$ (these may not be the only people in the region). We know the total population of each group in the region, and we know the populations of each group within subregions (or units). We have also assigned each subregion some index.

$$x_i := \text{Number of people of population X among subregion } i$$
$$x := \text{Vector of numbers of people of population X among each subregion} $$
$$y,z := \text{Similarly defined for people of population Y, Z,  respectively} $$
$$p_i := \text{Total population of subregion } i$$
$$\overline{p} := \sum_i p_i $$
$$\overline{x} := \sum_i x_i $$
$$A := \text{(Symmetric) adjacency matrix for the region} $$
$$I := \text{Identity matrix} $$
$$i \sim j := \text{(Unordered) pairs i,j when subregion i is adjacent to subregion j} $$
$$n := \text{Number of subregions (units), and implicit number of terms in a sum unless otherwise specified}$$
$$x_0 := \overline{x}/n$$
$$v := (x_1 - x_0,x_2 - x_0,\cdots, x_n - x_0)$$
$$|E| := \text{Number of edges, also half of the sum of the entries of A}$$

$$\text{Single brackets}(x,y) = \langle x,y \rangle := \sum_i x_iy_i + \sum_{i \sim j}x_iy_i + x_jy_i = x^T(A+I)y$$

$$\text{Skew}(x,y) := \frac{\langle x,x \rangle}{ \langle x,x \rangle + 2\langle x,y \rangle} $$

$$\text{Edge}(x,y) := \frac{1}{2}(\frac{\langle x,x \rangle}{ \langle x,x \rangle + 2\langle x,y \rangle} + \frac{\langle y,y \rangle}{ \langle y,y \rangle + 2\langle y,x \rangle}) $$

Note that edge can be extended to compare clustering of multiple sets, such as 

$$\text{Edge}(x,y,z) := \frac{1}{3}(\frac{\langle x,x \rangle}{ \langle x,x \rangle + 2\langle x,y \rangle  + 2\langle x,z \rangle } + \frac{\langle y,y \rangle}{ \langle y,y \rangle + 2\langle y,x \rangle + 2\langle y,z \rangle} + \frac{\langle z,z \rangle}{ \langle z,z \rangle + 2\langle x,z \rangle + 2\langle y,z \rangle} ) $$

$$\text{Skew'}(x,y) = \text{Skew prime}(x,y) :=  \frac{\langle x,x \rangle}{ \langle x,x \rangle + \langle x,y \rangle} $$

$$\text{HalfEdge}(x,y) := \frac{1}{2}(\frac{\langle x,x \rangle}{ \langle x,x \rangle + \langle x,y \rangle} + \frac{\langle y,y \rangle}{ \langle y,y \rangle + \langle y,x \rangle}) $$

HalfEdge can also be extended to multiple sets

$$\text{HalfEdge}(x,y,z) := \frac{1}{3}(\frac{\langle x,x \rangle}{ \langle x,x \rangle + \langle x,y \rangle  + \langle x,z \rangle } + \frac{\langle y,y \rangle}{ \langle y,y \rangle + \langle y,x \rangle + \langle y,z \rangle} + \frac{\langle z,z \rangle}{ \langle z,z \rangle + \langle x,z \rangle + \langle y,z \rangle} ) $$

$$\text{HalfEdgeInfinity}(x,y) = \text{HalfEdge}_{\infty}(x,y) := \frac{1}{2} (\frac{\sum x_i^2 }{\sum x_i(1+x_iy_i) } + \frac{\sum y_i^2 }{\sum y_i(1+x_iy_i) }) $$

$$\text{Dissimilarity}(x) := \frac{1}{2\overline{x}(\overline{p} - \overline{x})}\sum_i |x_i\overline{p} - p_i \overline{x}| $$

$$\text{Gini}(x) := \frac{1}{2\overline{x}(\overline{p} - \overline{x})}\sum_{i,j} |x_ip_j - p_i x_j|$$

$$\text{Moran's I}(x) := \frac{n}{|E|}\frac{\sum_{i \sim j}(x_i-x_0)(x_j - x_0)}{\sum_{i}(x_i-x_0)^2} =  \frac{n}{2|E|}\frac{v^TAv}{v^Tv} $$








Below is the script for the 6 tests: edge (and more_edge), half_edge (and more_half_edge), half_edge_infinity, moran's I, dissimilarity, and gini. Note that more_edge and more_half_edge are extensions of edge and half_edge, respectively, which allow for more than 2 demographic vectors (i.e. more_edge and more_half_edge are the same functions as edge and half_edge when fed in only 2 demographic vectors). 

The matrix A is the (symmetric) adjacency matrix for the region, and x and y are vectors with (integer) populations corresponding to each subregion. Both quantities are represented with numpy notation, for the ease of matrix multiplication. 

Note that there are a few helper functions:
Single brackets (noted as <x,y> in our literature) is one of the main building blocks, which is used in skew and skew_prime, which are in turn used in edge and half_edge (respectively). 
smart_symmetric_matrix_sum is a way to sum the entries of matrices, taking advantage of their symmetric structure (this is useful when the matrices are 1,000 by 1,000).

In dissimilarity and gini, we use a total_population_vector, which is a vector with the total populations of each subregion (note that this vector may be more than just x+y, if there are more than 2 demographics).

In [23]:
import numpy as np
import networkx as nx
import json
import csv
import matplotlib.pyplot as plt
import random

# <x,y> =  x^T(A+I)y = sum_i x_iy_i + sum_{i \sim j}x_iy_i + x_jy_i
def single_brackets(x, y, A):
    return np.matmul(x.T, np.matmul((A + np.identity(A[0].size)), y))

# <x,x> / (<x,x> + 2<x,y>)
def skew(x, y, A):
  x_prod = single_brackets(x, x, A)
  return float(x_prod) / float(x_prod + 2 * single_brackets(x, y, A))

def edge(x, y, A):
  # see more_edge for edge defined on more than 2 vectors
  skew_sum = 0
  for (a,b) in [(x,y), (y, x)]:
    skew_sum += skew(a, b, A)

  edge_result = float(skew_sum) / float(2)
  return edge_result

# more_edge is edge, handling more than 2 demographics
# dem_list is a list of vectors like x,y from previous functions
def more_edge(dem_list, A):
  n = len(dem_list)
  # first up, compute the single brackets (i,j) for 0 <= i <= j < n
  single_bracket_matrix = [[0 for i in range(n)] for j in range(n)]
  for i in range(n):
    for j in range(i,n):
      single_bracket_matrix[i][j] = single_brackets(dem_list[i], dem_list[j], A)
      # for ease of calling the values later,
      single_bracket_matrix[j][i] = single_bracket_matrix[i][j]

  # add up the terms that are generalizations of skew
  skew_sum = 0.
  for i in range(n):
    skew_denom = 0.
    for j in range(n):
      skew_denom += single_bracket_matrix[i][j]
    skew_denom *= 2.
    skew_denom -= single_bracket_matrix[i][i]
    skew_term = float((single_bracket_matrix[i][i])) / float(skew_denom)

    skew_sum += skew_term

  skew_sum /= float(n)
  return skew_sum

#<x,x> / (<x,x> + <x,y>)
def skew_prime(x, y, A):
  x_prod = single_brackets(x, x, A)
  return float(x_prod) / float(x_prod + single_brackets(x, y, A))

# average the skew primes
def half_edge(x, y, A):
  skew_sum = 0
  for (a,b) in [(x,y), (y, x)]:
    skew_sum += skew_prime(a, b, A)
  edge_result = float(skew_sum) / float(2)
  return edge_result

# more_half_edge is half_edge, handling more than 2 demographics
# dem_list is a list of vectors like x,y from previous functions
def more_half_edge(dem_list, A):
  n = len(dem_list)
  # first up, compute the single brackets (i,j) for all i,j in [n] x [n]
  single_bracket_matrix = [[0 for i in range(n)] for j in range(n)]
  for i in range(n):
    for j in range(i,n):
      single_bracket_matrix[i][j] = single_brackets(dem_list[i], dem_list[j], A)
      # for ease of calling the values later,
      single_bracket_matrix[j][i] = single_bracket_matrix[i][j]

  skew_p_sum = 0.
  for i in range(n):
    skew_p_denom = 0.
    for j in range(n):
      skew_p_denom += single_bracket_matrix[i][j]
    skew_p_term = float((single_bracket_matrix[i][i])) / float(skew_p_denom)

    skew_p_sum += skew_p_term

  skew_p_sum /= float(n)
  return skew_p_sum

def half_edge_infinity(x, y, A):
  x_square_sum = 0
  y_square_sum = 0
  first_denom = 0
  second_denom = 0
  n = len(x)
  for i in range(n):
    a = x[i]
    b = y[i]
    x_square_sum += a * a
    y_square_sum += b * b
    first_denom += a + a * a * b
    second_denom += b + b * a * b

  term1 = float(x_square_sum) / float(first_denom)
  term2 = float(y_square_sum) / float(second_denom)
  result = (term1 + term2) / 2.
  return result

# just runs over pairs i,j such that i<j, and sums the matrix, then multiplies by 2
def smart_symmetric_matrix_sum(A):
  n = len(A)
  accumulator = 0
  for i in range(n):
    for j in range(i + 1, n):
      accumulator += A[i,j]

  return float(2 * accumulator)

# moran's I
# n/(2|E|) * (v^TAv)/(v^Tv)
# where v is x, minus a vector with just mean of x as entries
def morans_I(x, A):
  n = float(len(x))
  A_sum = smart_symmetric_matrix_sum(A)
  v = x - np.average(x)
  # (v^TAv)
  r_num = float(np.matmul(v.T, np.matmul((A), v)))
  # (v^Tv)
  r_denom = float(np.matmul(v.T,  v))
  r_quotient = r_num / r_denom
  return (n / A_sum) * r_quotient

# total pop vector is a vector of the TOTAL population of each region, with same indices as x
def dissimilarity(x, total_pop_vector):
  total_pop = np.sum(total_pop_vector)
  x_pop = np.sum(x)
  n = len(x)

  first_term = 1. / float(2 * x_pop * (total_pop - x_pop))
  accumulator = 0.

  for i in range(n):
    accumulator += abs(x[i] * total_pop - total_pop_vector[i] * x_pop)
  return first_term * accumulator

# total_pop_vector similarly defined as before
def gini(x, total_pop_vector):
  total_pop = np.sum(total_pop_vector)
  x_pop = np.sum(x)
  n = len(x)

  first_term = 1. / float(2 * x_pop * (total_pop - x_pop))
  accumulator = 0.

  for i in range(n):
    for j in range(i + 1, n):
      accumulator += abs(x[i] * total_pop_vector[j] - total_pop_vector[i] * x[j])

  # the factor of 2 is necessary because we technically sum over all n^2 pairs (i,j), but I only did distinct pairs, and we ignore i=j
  return first_term * accumulator * 2

Below is a space to try out various configurations and print out the relevent quantities, using the function print_all_quantities. In a later section, we automate the process of creating adjacency matrices for grids. 

In [26]:
def print_all_quantities(x, y, A, total_pop_vector):
    print("")
    print("*** Computing scores... ***")
    print("x was " + str(x))
    print("y was " + str(y))
    print("single brackets(x, y, A): " + str(single_brackets(x, y, A)))
    print("skew(x, y, A): " + str(skew(x, y, A)))
    print("skew_prime(x, y, A): " + str(skew_prime(x, y, A)))
    print("The 6 scores:")
    print("edge(x, y, A): " + str(edge(x, y, A)))
    print("half_edge(x, y, A): " + str(half_edge(x, y, A)))
    print("half_edge_infinity(x, y, A): " + str(half_edge_infinity(x, y, A)))
    print("morans_I(x, A): " + str(morans_I(x, A)))
    print("dissimilarity(x, total_pop_vector): " + str(dissimilarity(x, total_pop_vector)))
    print("gini(x, total_pop_vector): " + str(gini(x, total_pop_vector)))
    print("")


# a -- b

x1 = np.array([2, 3])
y1 = np.array([5, 8])
A1 = np.array([[0,1], [1, 0]])
t1 = x1 + y1

# a -- b -- c

x2 = np.array([1, 6, 2])
y2 = np.array([5, 3, 8])
A2 = np.array([[0,1,0],[1,0,1],[0,1,0]])
t2 = x2 + y2

print_all_quantities(x1, y1, A1, t1)
print_all_quantities(x2, y2, A2, t2)

z2 = np.array([4, 7, 9])
print("more_edge on 3 vectors")
print(more_edge([x2, y2, z2],A2))
print("more_half_edge on 3 vectors")
print(more_half_edge([x2, y2, z2],A2))





*** Computing scores... ***
x was [2 3]
y was [5 8]
single brackets(x, y, A): 65.0
skew(x, y, A): 0.16129032258064516
skew_prime(x, y, A): 0.2777777777777778
The 6 scores:
edge(x, y, A): 0.36325385694249646
half_edge(x, y, A): 0.5
half_edge_infinity(x, y, A): 0.24152011319991912
morans_I(x, A): -1.0
dissimilarity(x, total_pop_vector): 0.015384615384615385
gini(x, total_pop_vector): 0.015384615384615385


*** Computing scores... ***
x was [1 6 2]
y was [5 3 8]
single brackets(x, y, A): 126.0
skew(x, y, A): 0.23404255319148937
skew_prime(x, y, A): 0.3793103448275862
The 6 scores:
edge(x, y, A): 0.3226287532312587
half_edge(x, y, A): 0.4810459008906143
half_edge_infinity(x, y, A): 0.352847824820919
morans_I(x, A): -0.9642857142857144
dissimilarity(x, total_pop_vector): 0.47916666666666663
gini(x, total_pop_vector): 0.4930555555555555

more_edge on 3 vectors
0.19927890840290519
more_half_edge on 3 vectors
0.3267282594971133


Below streamlines the process for testing on grids.

Note that the functions make_adj_matrix and print_matrix are helper functions for the function compute_values_on_grid, which is a way to streamline the process of testing the scores on n x m grids of your choice (in particular, it will create the adjacency matrix for you). All you need to do is feed in the number of rows and columns, and then the x and y vectors of length rows$*$cols. Then, on running it will print out a visual representation of the grid as well as the relevent scores.

In [22]:

# makes an adjacency matrix, found on https://stackoverflow.com/questions/16329403/how-can-you-make-an-adjacency-matrix-which-would-emulate-a-2d-grid
# the ordering of the entries is row by row (i.e. how you read a book. For a 2x3 matrix, you would read 1,2,3 - 4,5,6)
def make_adj_matrix(rows, cols):
    n = rows*cols
    M = [[0 for i in range(n)] for i in range(n)]
    for r in range(rows):
        for c in range(cols):
            i = r*cols + c
            # Two inner diagonals
            if c > 0: M[i-1][i] = M[i][i-1] = 1
            # Two outer diagonals
            if r > 0: M[i-cols][i] = M[i][i-cols] = 1
    return M

# prints out a matrix with a set number of rows, and columns, where x and y are population vectors of length rows*cols
def print_matrix(rows, cols, x, y):
    n = rows*cols
    # first, find out the longest number of digits in each
    xmax = np.amax(x)
    x_digits = len(str(xmax))
    ymax = np.amax(y)
    y_digits = len(str(ymax))

    for i in range(n):
        xfiller = " " * (x_digits - len(str(x[i])))
        yfiller = " " * (y_digits - len(str(y[i])))
        if (i % cols == 0):
            print("\n|", end="")
        print(str(x[i]) + xfiller + " , " + str(y[i]) + yfiller +  "|", end="")
    print("")
    print("")

# feed in a number of rows and columns, and vectors x and y (of length rows*cols), and print out values
# note that subregions are indexed as you would read them
# so a 2 row by 3 column matrix is indexed 1,2,3 (left to right on the top) then 4,5,6 (left to right on the bottom)
def compute_values_on_grid(rows, cols, x, y):
    print("Computing values on a " + str(rows) + " by " + str(cols) + " grid")
    A = np.array(make_adj_matrix(rows, cols))
    # again, this vector need not be x + y
    pop_vec = x + y
    print_matrix(rows, cols, x, y)
    print_all_quantities(x, y, A, pop_vec)

# as written, this is a very evenly distributed community (Moran's I cannot handle perfectly even)
print("almost evenly distributed community:")
x = np.array([100,102,100,100,106,100,100,100,100,95,100,100, 100, 100, 100, 100])
y = np.array([100,100,100,100,104,100,100,100,96,100,100,100, 100, 100, 98, 100])
compute_values_on_grid(4, 4, x, y)

print("isolated corner")
x = np.array([100,100,0,0,100,100,0,0,0,0,0,0, 0, 0, 0, 0])
y = np.array([0,0,100,100,0,0,100,100,100,100,100,100, 100, 100, 100, 100])
compute_values_on_grid(4, 4, x, y)

print("checkerboard")
x = np.array([100,0,100,0,0,100,0,100,100,0,100,0, 0, 100, 0, 100])
y = np.array([0,100,0,100,100,0,100,0,0,100,0,100, 100, 0, 100, 0])
compute_values_on_grid(4, 4, x, y)

# feel free to write your own configuration (it need not be a 4x4 grid)

almost evenly distributed community:
Computing values on a 4 by 4 grid

|100 , 100|102 , 100|100 , 100|100 , 100|
|106 , 104|100 , 100|100 , 100|100 , 100|
|100 , 96 |95  , 100|100 , 100|100 , 100|
|100 , 100|100 , 100|100 , 98 |100 , 100|


*** Computing scores... ***
x was [100 102 100 100 106 100 100 100 100  95 100 100 100 100 100 100]
y was [100 100 100 100 104 100 100 100  96 100 100 100 100 100  98 100]
single brackets(x, y, A): 639920.0
skew(x, y, A): 0.3338694272903053
skew_prime(x, y, A): 0.5006028633080613
The 6 scores:
edge(x, y, A): 0.33333794493376867
half_edge(x, y, A): 0.5000049496979694
half_edge_infinity(x, y, A): 0.009992785862252725
morans_I(x, A): 0.0009699321047526673
dissimilarity(x, total_pop_vector): 0.005461443148289698
gini(x, total_pop_vector): 0.007864634286307666

isolated corner
Computing values on a 4 by 4 grid

|100 , 0  |100 , 0  |0   , 100|0   , 100|
|100 , 0  |100 , 0  |0   , 100|0   , 100|
|0   , 100|0   , 100|0   , 100|0   , 100|
|0   , 100|0   , 1